In [ ]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.advanced_activations import ReLU

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [ ]:

def UNet():
    f = [32, 64, 128]
    inputs = keras.layers.Input((image_size, image_size, 1))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0])
    c2, p2 = down_block(p1, f[1]) 
    
    
    bn = bottleneck(p2, f[2])
    
    u1 = up_block(bn, c2, f[1]) 
    u2 = up_block(u1, c1, f[0])
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u2)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
image_size = 32
model = UNet()
model.compile(optimizer="SGD", loss="binary_crossentropy", metrics=["acc"])
model.summary()

In [ ]:
# Augment the images
# Notice that training, lab_training, validation, lav_validation are folders which contain another folder with the same name
# for more information see https://keras.io/preprocessing/image/
seed = 1
data_gen_args = dict(
                     rescale = 1./255, 
                     vertical_flip = True,
                     horizontal_flip=True)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

flow_from_dir_params = dict(target_size = (32,32), class_mode=None,shuffle=False, seed=seed, color_mode = 'grayscale', batch_size=32)

image_generator = image_datagen.flow_from_directory('./training_all', **flow_from_dir_params)

mask_generator = mask_datagen.flow_from_directory('./lab_training_all/', **flow_from_dir_params)
val_generator = image_datagen.flow_from_directory('./validation', **flow_from_dir_params)

val_mask_generator = mask_datagen.flow_from_directory('./lab_validation', **flow_from_dir_params)

# combine generators
train_generator = zip(image_generator, mask_generator)
valid_generator = zip(val_generator, val_mask_generator)


In [ ]:
model_train_val = model.fit_generator(generator=train_generator,validation_data=valid_generator,  epochs = 13, steps_per_epoch=800, validation_steps= 100, use_multiprocessing=False, max_queue_size=10, workers=1, verbose=1)

In [ ]:
accuracy = model_train_val.history['acc']
val_accuracy = model_train_val.history['val_acc']
loss = model_train_val.history['loss']
val_loss = model_train_val.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'C1', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('acc.png')

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'C1', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
#plt.show()
plt.savefig('loss.png')

In [ ]:
# Save model
from keras.models import model_from_json
from keras.models import load_model

model_json = model_train_val.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model_train_val.model.save_weights("model.h5")
print("Saved model to disk")